In [1]:
import os
import numpy as np
import pandas as pd

#Ruta base a las matrices de conectividad
base_path = "Matrices de conectividad"
output_base = "Matrices_z_transformadas"

#Poblaciones y parcellations
grupos = ["GK", "GW"]
parcellations = ["schaefer_400ROI", "seitzman_300ROI"]
medida = "correlation"  # Solo trabajaremos sobre correlaciones

#Función para aplicar la Fisher Z con estabilización
def apply_fisher_z_stable(df):
    eps = 1e-9  #pequeño valor para evitar correlación perfecta
    mat = df.to_numpy(dtype=np.float64)
    mat = np.clip(mat, a_min=-1 + eps, a_max=1 - eps)  #Evita ±1 exactos (codigo pasado por keveen)
    z_mat = np.arctanh(mat)
    #np.fill_diagonal(z_mat, 0.0)  # Poner la diagonal en cero
    #z_mat = np.nan_to_num(z_mat, nan=0.0, posinf=0.0, neginf=0.0)  # Limpia posibles NaN o inf
    return pd.DataFrame(z_mat)

#Recorremos cada combinación de grupo y parcellation 
for grupo in grupos:
    for parc in parcellations: #le vamos a aplicar la z a todo
        parc_folder = f"{grupo}_{parc}"
        input_dir = os.path.join(base_path, parc_folder, str(parc.split("_")[0]), medida)
        output_dir = os.path.join(output_base, parc_folder, medida)
        
        for window in sorted(os.listdir(input_dir)):  #Recorremos cada ventana
            window_input_path = os.path.join(input_dir, window)
            window_output_path = os.path.join(output_dir, window)
            os.makedirs(window_output_path, exist_ok=True)

            for filename in os.listdir(window_input_path):
                if filename.endswith(".csv"):  #Un archivo por sujeto y por ventana
                    filepath = os.path.join(window_input_path, filename)
                    df = pd.read_csv(filepath, header=None)

                    #Aplicamos la Fisher Z corregida
                    df_z = apply_fisher_z_stable(df)

                    #se Guarda el resultado
                    output_path = os.path.join(window_output_path, filename)
                    df_z.to_csv(output_path, header=False, index=False)

print("Transformación Fisher Z completada y guardada en:", output_base)


C:\Users\ADMH2\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Transformación Fisher Z completada y guardada en: Matrices_z_transformadas


In [ ]:
import os
import numpy as np
import pandas as pd

def vectorize_matrix(matrix):
    """
    Convierte una matriz simétrica en un vector unidimensional tomando solo
    la triangular superior sin la diagonal, fila por fila.
    """
    triu_indices = np.triu_indices_from(matrix, k=1) #devuelve los índices del triángulo superior de arr.
    return matrix[triu_indices]

def procesar_grupo_parcellation(input_base, grupo, parcellation, medida, n_ventanas):
    """
    Procesa todas las matrices de un grupo-parcellation y retorna una tabla 2D
    con un sujeto por fila y una concatenación de ventanas como columnas.
    """
    carpeta = f"{grupo}_{parcellation}"
    matriz_sujetos = []

    for subj_index in range(1000):  #número alto para parar cuando no existan más archivos
        vector_concatenado = []
        sujeto_existe = True

        for ventana_idx in range(n_ventanas): #recorre las diferentes ventanas extrayendo el mismo sujeto
            ventana = f"w_{ventana_idx:02d}"
            archivo = f"subj_{subj_index}_w_{ventana_idx:02d}_connectome_measure_{medida}.csv"
            ruta_archivo = os.path.join(
                input_base, carpeta, medida, ventana, archivo
            )

            if not os.path.exists(ruta_archivo):
                sujeto_existe = False
                break
            print(ruta_archivo)
            matriz = pd.read_csv(ruta_archivo, header=None).values #aqui lee el csv del sujeto
            vector = vectorize_matrix(matriz)#aqui convierte esa matriz en vector
            vector_concatenado.extend(vector) #aqui va concatenando las ventanas una atras de la otra

        if sujeto_existe:
            matriz_sujetos.append(vector_concatenado) #aqui se va armando la data completa
        else:
            break  #No hay más sujetos

    columnas = len(matriz_sujetos[0]) if matriz_sujetos else 0
    df_final = pd.DataFrame(matriz_sujetos, columns=[f"c_{i}" for i in range(columnas)])#aqui se arma el dataframe y se le pone un nombre a las columnas
    df_final["grupo"] = grupo  #etiqueta para ML

    return df_final


In [ ]:
input_base = "Matrices_z_transformadas"
medida = "correlation"
n_ventanas = 11

#procesar ambos grupos y unirlos
df_gk = procesar_grupo_parcellation(input_base, "GK", "schaefer_400ROI", medida, n_ventanas)
df_gw = procesar_grupo_parcellation(input_base, "GW", "schaefer_400ROI", medida, n_ventanas)

#Concatenar en un solo DataFrame
df_final = pd.concat([df_gk, df_gw], ignore_index=True)

#guardado
df_final.to_csv("dataset_vectorizado_para_ML.csv", index=False)
print("Dataset 2D generado y guardado como 'dataset_vectorizado_para_ML.csv'")


Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_00\subj_0_w_00_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_01\subj_0_w_01_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_02\subj_0_w_02_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_03\subj_0_w_03_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_04\subj_0_w_04_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_05\subj_0_w_05_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_06\subj_0_w_06_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_07\subj_0_w_07_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_schaefer_400ROI\correlation\w_08\subj_0_w_08_connectome_measure_correlation.csv
Matrices_z_transformadas\GK_

In [ ]:
import pandas as pd

csv_path = "dataset_vectorizado_para_ML.csv"

df = pd.read_csv(csv_path)
print(f"El archivo tiene {df.shape[0]} filas (sujetos) y {df.shape[1]} columnas (características + columna de grupo).")


El archivo tiene 57 filas (sujetos) y 877801 columnas (características + columna de grupo).
